<h2>OBSERVATIONAL ASTROPHYSICS – FALL 2019 Reduction Exercise: Part 2</h2>

In Part 1 you constructed images that were overscan subtracted and trimmed.

In Part 2 you will will:

1. combine your bias frames to make a master bias frame;
2. bias subtract all your images;
3. make a combined flatfield image in each band;
4. flatfield the flatfield images and science images;

Remember that your raw data are stored in ~/RFSLAB/USER_DPT/_PUBLIC/ASTR596/Data/Raw/< your night>
and your reduced images are stored in ~/RFSLAB/USER_DPT/< KUID>/ASTR596/Data/Reduced/< your_night> 

In [7]:
import numpy as np
from astropy import units as u
from astropy.nddata import CCDData
import ccdproc
from matplotlib import pyplot as plt
from ccdproc import Combiner
from astropy.io import ascii

You will first read in bias images and combine them using the "Combiner" task described at https://ccdproc.readthedocs.io/en/latest/image_combination.html

Make sure to read the section on sigma-clipping, which describes how we generate maskes for comining and eventually combine the images.  

As a first step you will need to make a set of lists that contain files.  You can name the lists what you'd like, e.g. biaslist, but you'll need to make sure that they are in the right places in the program:
1. biaslist: all of your good trimmed bias frames
2. flatlist_< band>: all of the trimmed flatfields corresponding to a given band
3. science_< band>_< target>: all of the science frames for each target in each band

In [8]:
rawpath = "/home/l943z843/ASTR596/Data/Raw/20191106/"
redpath = "/home/l943z843/RFSLAB/USER_DPT/l943z843/ASTR596/Data/Reduced2/20191106/"

biaslist = "biaslist.txt"
biaslistpath = redpath + biaslist


#initialize list of bias frames
bias_imlist = []
#this way of opening the file ensures that it closes after the loop is done.
with open(biaslistpath,'r') as fp:
    #read first line
    line = fp.readline()

    #build the combine list of all images
    #read every subsequent line
    while line:
        #this removes the trailing newline charactter and converts the output list to a scalar
        imname = line.split()
        print('1',imname)

        if (imname):
            imname = imname[0]

            #create the image name, including the path
            imstr = redpath + imname
            #read that into a CCDData object.  This allows you to specify a unit
            im = CCDData.read(imstr)

            #make a list of all the CCDData instances of each image
            bias_imlist = bias_imlist + [im]

        line = fp.readline()

print('combining images')
#Combiner list of all bias images
bias_comb = Combiner(bias_imlist)

#now create masks with sigma clipping algorithm
#This creates a mask for each image that is the size of each image but which has 0 values
#everywhere except those pixels in excess of low_thresh and high_thresh sigma from the median.
#Those pixels get a value of 1
#*************
#given the number of pixels in your image how must you set high_thresh to avoid flagging more than
#about 5 pixels because of expected statistical variations.  Assume the noise is distributed like a Gaussian.
#Remember, you want to get rid of really deviant pixels, so if you set the threshold too high you won't
#get rid of anything!
bias_comb.sigma_clipping(low_thresh=3, high_thresh=5, func=np.ma.median)

#Use these masks to combine the images.  This now becomes your master bias
master_bias = bias_comb.average_combine()

#now write out the master bias
masterbiaspath = redpath + 'master_bias.fits'
master_bias.write(masterbiaspath,overwrite=True)


1 ['b000_tr.fits']
1 ['b001_tr.fits']
1 ['b002_tr.fits']
1 ['b003_tr.fits']
1 ['b004_tr.fits']
1 ['b005_tr.fits']
1 ['b006_tr.fits']
1 ['b007_tr.fits']
1 ['b009_tr.fits']
1 ['b010_tr.fits']
1 ['b012_tr.fits']
1 ['b013_tr.fits']
1 ['b014_tr.fits']
1 ['b015_tr.fits']
1 ['b016_tr.fits']
1 ['b017_tr.fits']
1 ['b018_tr.fits']
1 ['b020_tr.fits']
1 []
combining images


What follows is a function to subtract bias frame from a list of images

In [9]:
def bias_sub(imlist_name, redpath, master_bias):
    #go through each image in a list, subtract the master bias from that image, and write the image out again.
    #imlist_name: the name of the file (without path) that contains the list of images to subtract the bias from
    #redpath: the path of directory with your reduced data
    #master_bias: the actual CCDData master bias, *not* the filename of the master bias fits file

    imlistpath= redpath + imlist_name
    with open(imlistpath,'r') as fp:
        #read first line
        line = fp.readline()

        #read every subsequent line
        while line:
            #read in each file
            imname = line.split()
            if (imname):
                imname = imname[0]
                impath = redpath + imname
                #load file into CCDData objet
                im = CCDData.read(impath)

                #***********  
                #insert appropriate expression from https://ccdproc.readthedocs.io/en/latest/image_combination.html
                #subtract bias frame
                C = Combiner([im])
                im_bsub = ccdproc.subtract_bias(im, master_bias)
                #now write out the file, replacing ".fits" with "b.fits"
                bsub_imname = imname.replace('.fits','b.fits',1)
                print(bsub_imname)
                bsub_path = redpath + bsub_imname
                im_bsub.write(bsub_path,overwrite=True)

            line = fp.readline()



now subtract the bias frames from all images

In [10]:
#**************
#put in the appropriate file name for each of the flats and science frames
#subtract bias from flats in each band (you will need two calls to this function)
bias_sub("flatlist_B.txt", redpath, master_bias)
bias_sub("flatlist_V.txt", redpath, master_bias)
bias_sub("flatlist_R.txt", redpath, master_bias)
#subtract bias from science frames in each band.  You will need to do this for each band and each target
bias_sub("sciencelist.txt",redpath,master_bias)


f026_trb.fits
f027_trb.fits
f028_trb.fits
f029_trb.fits
f030_trb.fits
f031_trb.fits
f032_trb.fits
f033_trb.fits
f034_trb.fits
f035_trb.fits
f036_trb.fits
f037_trb.fits
f038_trb.fits
s049_trb.fits
s050_trb.fits
s051_trb.fits
s052_trb.fits
s053_trb.fits
s054_trb.fits
s055_trb.fits
s056_trb.fits
s057_trb.fits
s058_trb.fits
s064_trb.fits
s065_trb.fits
s066_trb.fits
s067_trb.fits
s068_trb.fits
s069_trb.fits
s070_trb.fits
s071_trb.fits
s072_trb.fits
s073_trb.fits
s074_trb.fits
s078_trb.fits
s079_trb.fits
s080_trb.fits
s081_trb.fits
s082_trb.fits
s083_trb.fits
s084_trb.fits
s085_trb.fits
s086_trb.fits
s087_trb.fits
s088_trb.fits


Now combine the flatfields into master flats.  In this process you will scale each object to their mean before combining.  

Here we will use a median combination instead of a sigma clipping combination.  In the cell below please describe why this is a good idea.

<h4>Your answer here:</h4>   The median will flatten the image, sigma clipping would leave room for outliers.

Towards the bottom of this routine we will be using a "Lambda" function to normalize our images.  These are small anonymous throw-away functions that are only used once in a code.  Their use is described here https://www.python-course.eu/python3_lambda.php.  

In [16]:
def master_flatmake(imlist_name, redpath, master_flatname):
    #a function that makes a master flatfield, taking its flats from a list.  
    #The process will be very similar to that for bias frames, though we 
    #1) normalize the frames before combining and 
    #2) make it a function as we may need to generate flats for mutliple bands

    #imlist_name: the name of the file (without path) that contains the list of images that will be combined
    #redpath: the path of directory with your reduced data
    #master_flatname: the name of the combined master flatfield

    imlistpath = redpath + imlist_name


    #initialize list of flat frames
    flat_imlist = []
    #this way of opening the file ensures that it closes after the loop is done.
    with open(imlistpath,'r') as fp:
        #read first line
        line = fp.readline()

        #build the combine list of all images
        #read every subsequent line
        while line:
            #this removes the trailing newline charactter and converts the output list to a scalar
            imname = line.split()
            if (imname):
                imname = imname[0]

                #create the image name, including the path
                imstr = redpath + imname
                #read that into a CCDData object.  This allows you to specify a unit
                im = CCDData.read(imstr)

                #make a list of all the CCDData instances of each image
                flat_imlist = flat_imlist + [im]

            line = fp.readline()

    #Combiner list of all flat images
    flat_combiner = Combiner(flat_imlist)
    
    #######################################
    #this is an anonymous throw away function that takes an argument and is intended
    #to be used just once.  In this case, the scaling functionality of the combiner is 
    #set to scale by the inverse of the mean, so images with high counts get scaled down and
    #images wtih low counts get scaled up
    scaling_func = lambda arr: 1/np.ma.average(arr)
    flat_combiner.scaling = scaling_func

    #combine the scaled images
    #****************
    #insert appropriate expression from https://ccdproc.readthedocs.io/en/latest/image_combination.html
    master_flat = flat_combiner.median_combine()
    

    #now write out the master flat
    masterflatpath = redpath + master_flatname
    master_flat.write(masterflatpath,overwrite=True)


Now use this function to generate master flats

In [17]:
#***********
#make the appropriate calls to master_flatmake to make the flatfields. 
#Do this for each of your sets of flatfields
#figure out what the missing argument is
master_flatmake("flatlist_B.txt", redpath, 'master_flat_B.fits')
master_flatmake("flatlist_V.txt", redpath, 'master_flat_V.fits')
master_flatmake("flatlist_R.txt", redpath, 'master_flat_R.fits')

Write in the cell below how you can assess whether or not you have a good master flatfield.  What characteristics might you look for and does your master flatfield image have them.  Write also how your master flat differs from the individual flatfield images.

<h4>Your answer goes here:</h4> If it's a flat image. It will be uniform and not have any random stars or anything, it will generally look flat. This is what my master flats look like. The individual flatfield images will have non flat sections such as cosmic rays since they have not bee corrected for yet.

-------------------------------
Now we will flatfield all of our science and flatfield images.  This is similar to the process for bias subtraction except that we will be dividing by the master flat, as opposed to subtracting the master bias.

In [34]:
def flat_corr(imlist_name, redpath, masterflat_name):
    #go through each image in a list, subtract the master bias from that image, and write the image out again.
    #imlist_name: the name of the file (without path) that contains the list of images to subtract the bias from
    #redpath: the path of directory with your reduced data
    #master_name: the filename of the master flatfield

    masterflat_path = redpath + masterflat_name
    master_flat = CCDData.read(masterflat_path)
    
    imlistpath= redpath + imlist_name
    with open(imlistpath,'r') as fp:
        #read first line
        line = fp.readline()

        #read every subsequent line
        while line:
            #read in each file
            imname = line.split()
            if (imname):
                imname = imname[0]
                impath = redpath + imname
                #load file into CCDData objet
                im = CCDData.read(impath)
            
                #***********  
                #insert appropriate expression from https://ccdproc.readthedocs.io/en/latest/image_combination.html
                #subtract bias frame
                C = Combiner([im])
                im_flat = ccdproc.subtract_bias(im, master_flat)

                #now write out the file
                flat_imname = imname.replace('.fits','f.fits',1)

                imflat_path = redpath + flat_imname
                im_flat.write(imflat_path,overwrite=True)
            line = fp.readline()


In [35]:
#*************
#flatfield correct all the flats
#put in the correct argument for the function
flat_corr("flatlist_B.txt", redpath, 'master_flat_B.fits')
flat_corr("flatlist_V.txt", redpath, 'master_flat_V.fits')
flat_corr("flatlist_R.txt", redpath, 'master_flat_R.fits')

#flatfield correct the science frames in each band
flat_corr("sciencelist.txt",redpath, 'master_flat_B.fits')
flat_corr("sciencelist.txt",redpath, 'master_flat_V.fits')
flat_corr("sciencelist.txt",redpath, 'master_flat_R.fits')

**write in this cell how you can tell if your master flatfield in each band did a good job.  Be quantitative if possible.  Also write how the flatfielded science images look different from those before flatfielding.**

Answer: You can check whether your final image is flat. I can't really see a difference in them before and after but I'm pretty sure I did it right. 